In [ ]:
!pip install tkinter
!apt-get update
!apt-get install -y xvfb
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
from PIL import Image, ImageTk

# Function to correct turbulence using a simple deblurring approach
def correct_turbulence(frame):
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])  # Sharpening filter
    corrected_frame = cv2.filter2D(frame, -1, kernel)
    return corrected_frame

# Function to detect moving objects
def detect_moving_objects(frame, background_subtractor):
    fgmask = background_subtractor.apply(frame)
    return fgmask

# GUI to select video file
def select_file():
    file_path = filedialog.askopenfilename(filetypes=[("Video Files", "*.mp4;*.avi;*.mov")])
    if file_path:
        process_video(file_path)
    else:
        messagebox.showerror("Error", "No file selected.")

# Function to verify X11 and start a virtual display
def verify_and_start_display():
    """
    Checks if $DISPLAY is set, and starts a virtual X11 display if needed.
    """
    if not os.environ.get('DISPLAY'):
        import subprocess
        print("Starting Xvfb virtual display...")  # Message before process
        # Start Xvfb and wait for it to be ready
        process = subprocess.Popen(['Xvfb', ':99', '-screen', '0', '1024x768x24'])

        # Give Xvfb some time to start up
        import time
        time.sleep(2) # add a short sleep to ensure Xvfb has started

        os.environ['DISPLAY'] = ':99'  # Setup $DISPLAY variable
        print("Xvfb started and DISPLAY set.")

# Process the video
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        messagebox.showerror("Error", "Cannot open video file.")
        return

    fgbg = cv2.createBackgroundSubtractorMOG2()
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        corrected_frame = correct_turbulence(frame)
        fgmask = detect_moving_objects(corrected_frame, fgbg)

        cv2.imshow("Original Video", frame)
        cv2.imshow("Corrected Video", corrected_frame)
        cv2.imshow("Moving Objects", fgmask)

        if cv2.waitKey(30) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

# GUI Application
def create_gui():
    verify_and_start_display()
    root = tk.Tk()
    root.title("Turbulence Correction & Moving Object Detection")

    label = tk.Label(root, text="Select a video file to process:", font=("Arial", 12))
    label.pack(pady=10)

    button = tk.Button(root, text="Select Video", command=select_file, font=("Arial", 12))
    button.pack(pady=10)

    root.mainloop()

# Run GUI
if __name__ == "__main__":
    create_gui()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# Function to correct turbulence using a sharpening filter
def correct_turbulence(frame):
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    corrected_frame = cv2.filter2D(frame, -1, kernel)
    return corrected_frame

# Function to detect moving objects
def detect_moving_objects(frame, background_subtractor):
    fgmask = background_subtractor.apply(frame)
    return fgmask

# Function to process the video and display frames in Jupyter Notebook
def process_video(video_path, save_output=False):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Cannot open video file.")
        return

    fgbg = cv2.createBackgroundSubtractorMOG2()

    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    output_video = None

    if save_output:
        output_video = cv2.VideoWriter('output.avi', cv2.VideoWriter_fourcc(*'XVID'), 20, (frame_width, frame_height))

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        corrected_frame = correct_turbulence(frame)
        fgmask = detect_moving_objects(corrected_frame, fgbg)

        # Convert frame to RGB (for displaying in Jupyter)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        fgmask_rgb = cv2.cvtColor(fgmask, cv2.COLOR_GRAY2RGB)

        # Plot results
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(frame_rgb)
        plt.title("Original Video")
        plt.axis("off")

        plt.subplot(1, 2, 2)
        plt.imshow(fgmask_rgb)
        plt.title("Moving Objects Detected")
        plt.axis("off")

        plt.show()

        if save_output:
            output_video.write(corrected_frame)

        frame_count += 1
        if frame_count > 100:  # Stop early to prevent long execution in Jupyter
            break

    cap.release()
    if save_output:
        output_video.release()
    print("Processing complete.")

# Example usage
video_path = "Sample_Video.mp4"  # Ensure this file exists in the working directory
if os.path.exists(video_path):
    process_video(video_path, save_output=True)
else:
    print("Error: Video file not found.")